# Analyze the model performance

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import train_test_split

import pandas as pd 
import numpy as np 
import joblib

In [4]:
model = joblib.load("model.joblib")

In [5]:
scaler = joblib.load("scaler.joblib")